In [2]:
import logging
from getpass import getpass

import mysql.connector
from mysql.connector import Error

def connect_to_database():
    try:
        con = mysql.connector.connect(
            host='localhost',
            user='root',
            password='Justin@2021',
            database='music_library'
        )
        print("Connected to the database.")
        return con
    except Error as e:
        print(f"Error connecting to the database: {e}")
        return None

def login(con):
    username = input("Please enter your username: ")
    password = getpass("Please enter your password: ")

    cursor = con.cursor()
    query = "SELECT * FROM user WHERE username = %s AND password = %s"
    cursor.execute(query, (username, password))
    result = cursor.fetchone()

    if result is None:
        print("Invalid username or password. Please try again.")
        return None
    else:
        stored_username = username
        return stored_username
    
def add_artist(con, result):
    if result is None:
        print("You need to log in to add an artist.")
    else:
        name = input("Enter the artist name: ")

        try:
            cursor = con.cursor()
            query = "INSERT INTO artist (name) VALUES (%s)"
            cursor.execute(query, (name,))
            con.commit()
            nextcommand = input("Artist added successfully! What would you like to do next?")
            cursor.close()
        except Error as e:
            if e.errno == 1062:  # Duplicate entry error code
                print("Error: Artist already exists in the database.")
            else:
                print(f"Error: {e}")
                
def add_song(con, result):
    if result is None:
        print("You need to log in to add a song.")
    else:
        title = input("Enter the song title: ")
        artist = input("Enter the artist name: ")

        cursor = con.cursor()
        query = "INSERT INTO song (title, artist) VALUES (%s, %s)"
        cursor.execute(query, (title, artist))
        con.commit()
        cursor.close()

        nextcommand = input("Song added successfully! What would you like to do next?")

def add_album(con, result):
    if result is None:
        print("You need to log in to add an album.")
    else:
        title = input("Enter the album title: ")
        artist = input("Enter the artist name: ")
        release_date = input("Enter the release date: ")

        cursor = con.cursor()
        query = "INSERT INTO album (title, artist, release_date) VALUES (%s, %s, %s)"
        cursor.execute(query, (title, artist, release_date))
        con.commit()

        nextcommand = input("Album added successfully! What would you like to do next?")
        cursor.close()

def show_songs(con, result):
    if result is None:
        print("You need to log in to display songs.")
    else:
        cursor = con.cursor()
        query = "SELECT * FROM song"
        cursor.execute(query)

        songs = cursor.fetchall()
        for song in songs:
            print(song)

        nextcommand = input("Songs shown above. What would you like to do next?")
        cursor.close()

def show_artists(con, result):
    if result is None:
        print("You need to log in to display artists.")
    else:
        cursor = con.cursor()
        query = "SELECT * FROM artist"
        cursor.execute(query)

        artists = cursor.fetchall()
        for artist in artists:
            print(artist)

        nextcommand = input("Artists shown above. What would you like to do next?")
        cursor.close()

def show_albums(con, result):
    if result is None:
        print("You need to log in to display albums.")
    else:
        cursor = con.cursor()
        query = "SELECT * FROM album"
        cursor.execute(query)

        albums = cursor.fetchall()
        for album in albums:
            print(album)

        nextcommand = input("Albums shown above. What would you like to do next?")
        cursor.close()

def show_playlists(con, result):
    if result is None:
        print("You need to log in to display playlists.")
    else:
        username = result  # Assuming the username is the first column in the result
        cursor = con.cursor()
        playlist_query = "SELECT * FROM playlist WHERE username = %s"
        cursor.execute(playlist_query, (username,))
        playlists = cursor.fetchall()

        if len(playlists) == 0:
            print("No playlists found for the user.")
        else:
            for playlist in playlists:
                print(playlist)

        nextcommand = input("Playlists shown above. What would you like to do next?")
        cursor.close()
        
def create_playlist(con, result):
    if result is None:
        print("You need to log in to create a playlist.")
    else:
        username = result

        # Get the playlist name from the user
        playlist_name = input("Enter the playlist name: ")

        try:
            # Call the stored procedure
            cursor = con.cursor()
            cursor.callproc("CreatePlaylist", [username, playlist_name])
            con.commit()
            cursor.close()

            print("Playlist created successfully!")
        except mysql.connector.Error as e:
            print(f"Error creating playlist: {e}")
            
def add_song_to_playlist(con, result):
    if result is None:
        print("You need to log in to add a song to a playlist.")
    else:
        username = result

        # Get the playlist name from the user
        playlist_name = input("Enter the playlist name: ")
        # Get the song title and artist from the user
        title = input("Enter the song title: ")
        artist = input("Enter the artist name: ")

        try:
            # Call the stored procedure
            cursor = con.cursor()
            cursor.callproc("AddSongToPlaylist", [username, playlist_name, title, artist])
            con.commit()
            cursor.close()

            print("Song added to the playlist successfully!")
        except mysql.connector.Error as e:
            print(f"Error adding song to playlist: {e}")
            
def remove_song_from_playlist(con, result):
    if result is None:
        print("You need to log in to delete a song from a playlist.")
    else:
        username = result

        # Get the playlist name from the user
        playlist_name = input("Enter the playlist name: ")
        # Get the song title and artist from the user
        title = input("Enter the song title: ")
        artist = input("Enter the artist name: ")

        try:
            # Call the stored procedure
            cursor = con.cursor()
            cursor.callproc("DeleteSongFromPlaylist", [username, playlist_name, title, artist])
            con.commit()
            cursor.close()

            print("Song deleted from the playlist successfully!")
        except mysql.connector.Error as e:
            print(f"Error deleting song from playlist: {e}")
            
def delete_song(con, result):
    if result is None:
        print("You need to log in to delete a song.")
    else:
        title = input("Enter the song title: ")
        artist = input("Enter the artist name: ")

        try:
            cursor = con.cursor()
            query = "DELETE FROM song WHERE title = %s AND artist = %s"
            cursor.execute(query, (title, artist))
            con.commit()
            nextcommand = input("Song deleted successfully! What would you like to do next?")
            cursor.close()
        except Error as e:
            print(f"Error: {e}")

def delete_album(con, result):
    if result is None:
        print("You need to log in to delete an album.")
    else:
        title = input("Enter the album title: ")
        artist = input("Enter the artist name: ")

        try:
            cursor = con.cursor()
            query = "DELETE FROM album WHERE title = %s AND artist = %s"
            cursor.execute(query, (title, artist))
            con.commit()
            nextcommand = input("Album deleted successfully! What would you like to do next?")
            cursor.close()
        except Error as e:
            print(f"Error: {e}")

            
def update_playlist_name(con, result):
    if result is None:
        print("You need to log in to update a playlist name.")
    else:
        current_name = input("Enter the current playlist name: ")
        new_name = input("Enter the new playlist name: ")

        try:
            cursor = con.cursor()
            query = "UPDATE playlist SET name = %s WHERE username = %s AND name = %s"
            cursor.execute(query, (new_name, result, current_name))
            con.commit()
            nextcommand = input("Playlist name updated successfully! What would you like to do next?")
            cursor.close()
        except Error as e:
            print(f"Error: {e}")






con = connect_to_database()
if con is None:
    raise SystemExit

stored_username = login(con)

while True:
    print("Dashboard")
    print("1. Add an artist")
    print("2. Add a song")
    print("3. Add an album")
    print("4. Show songs")
    print("5. Show artists")
    print("6. Show albums")
    print("7. Show playlists")
    print("8. Create a playlist")
    print("9. Add Song to playlist")
    print("10. Remove Song from playlist")
    print("11. Delete song")
    print("12. Delete album")
    print("13. Update playlist name")
    print("14. Log out")

    option = input("Select an option: ")


    if option == "1":
        add_artist(con, stored_username)
    elif option == "2":
        add_song(con, stored_username)
    elif option == "3":
        add_album(con, stored_username)
    elif option == "4":
        show_songs(con, stored_username)
    elif option == "5":
        show_artists(con, stored_username)
    elif option == "6":
        show_albums(con, stored_username)
    elif option == "7":
        show_playlists(con, stored_username)
    elif option == "8":
        create_playlist(con, stored_username)
    elif option == "9":
        add_song_to_playlist(con, stored_username)
    elif option == "10":
        remove_song_from_playlist(con, stored_username)
    elif option == "11":
        delete_song(con, stored_username)
    elif option == "12":
        delete_album(con, stored_username)
    elif option == "13":
        update_playlist_name(con, stored_username)
    elif option == "14":
        print("Logged out.")
        break;
    else:
        print("Invalid option.")


con.close()

Connected to the database.
Please enter your username: justin
Please enter your password: ········
Dashboard
1. Add an artist
2. Add a song
3. Add an album
4. Show songs
5. Show artists
6. Show albums
7. Show playlists
8. Create a playlist
9. Add Song to playlist
10. Remove Song from playlist
11. Delete song
12. Delete album
13. Update playlist name
14. Log out
Select an option: 7
('justin', 'Bops')
Playlists shown above. What would you like to do next?8
Dashboard
1. Add an artist
2. Add a song
3. Add an album
4. Show songs
5. Show artists
6. Show albums
7. Show playlists
8. Create a playlist
9. Add Song to playlist
10. Remove Song from playlist
11. Delete song
12. Delete album
13. Update playlist name
14. Log out
Select an option: 8
Enter the playlist name: Summertime
Playlist created successfully!
Dashboard
1. Add an artist
2. Add a song
3. Add an album
4. Show songs
5. Show artists
6. Show albums
7. Show playlists
8. Create a playlist
9. Add Song to playlist
10. Remove Song from pla